# Magnetic-Inspired Classification Algorithm
is a unique and dynamic approach to classifying data
points based on
magnetic forces of attraction and repulsion between class centers (magnetic poles).
Inspired by the physical properties of magnets, the algorithm adapts and refines its classification boundaries by adjusting the centroids
(class centers)using magnetic principles.
 It introduces an iterative process where misclassified points are sent back
to be reconsideredby the model until convergence or a predefined number of epochs

#Key Concepts:
1. Magnetic Layer Classification (Initial Classification)
2. Secondary Decision Layers for Repulsive and Attractive Layers
3. Epoch-Based Reconsideration
4. Magnetic Force Refinement (Polarity and Flux)
5. Inter-Class Interaction (Handling Class Overlap)
6. Final Refinement and Convergence

#Imports

In [30]:
import numpy as np
from sklearn.datasets import make_classification, make_blobs
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC   #comparision purpose
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# The MagneticInspiredClassifier is a custom classification algorithm that simulates magnetic forces (attraction and repulsion)
# between data points and class centers (magnetic poles). The core idea is to treat class centers as magnetic poles and
# data points as charged particles, where each class center attracts data points from its respective class and repels points
# from other classes. This force-based classification method iteratively refines the class centers (centroids) using feedback
# from misclassified points. The algorithm applies a multi-layer classification approach, where data points are classified
# initially, and misclassified points are reconsidered in subsequent epochs. The class centroids are adjusted based on these
# reconsiderations to improve the decision boundaries.
#
# Key features:
# 1. **Magnetic Forces**: The algorithm uses both **attractive** forces to pull points toward their correct class centers
#    and **repulsive** forces to push points away from incorrect class centers.
# 2. **Epoch-Based Refinement**: Misclassified points are sent back for reconsideration, and the class centers are adjusted
#    iteratively, refining the decision boundaries over multiple epochs.
# 3. **Centroid Adjustment**: The centroids are updated to minimize misclassifications, moving closer to correctly classified
#    points and away from misclassified ones.
# 4. **Flexibility**: The algorithm can adaptively refine decision boundaries based on the data, offering flexibility in classification.
#
# Limitations:
# - The algorithm may struggle with non-linearly separable datasets unless additional methods (like repulsion between class
#   centers) are introduced to manage complex boundaries.
# - Its performance may also depend on the choice of parameters such as learning rate, number of epochs, and magnetic strength.
#
# In summary, the MagneticInspiredClassifier provides a novel and dynamic approach to classification, leveraging
# magnetic-like forces to adapt and refine the decision boundaries over time, improving accuracy through iterative reconsideration
# of misclassified points.


In [5]:
# Base Class for Magnetic-Inspired Classifier
class MagneticInspiredClassifier:
    def __init__(self, n_classes, max_iter=100, epoch=3, learning_rate=0.1, magnetic_strength=1.0):
        """
        Initialize the classifier with the number of classes and model parameters.

        n_classes: Number of classes in the classification problem
        max_iter: Maximum number of iterations for training
        epoch: Number of epochs (reconsideration cycles)
        learning_rate: Controls how much the centroids move during each update
        magnetic_strength: The strength of attraction or repulsion between data points and class centers
        """
        self.n_classes = n_classes
        self.max_iter = max_iter
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.magnetic_strength = magnetic_strength
        self.centroids = None  # Store the class centers

    def fit(self, X, y):
        """
        Fit the model to the data, adjusting the class centers based on misclassified points.

        X: Training data features
        y: Training data labels
        """
        # Initialize class centers (magnetic poles) as the mean of points in each class
        self.centroids = np.array([X[y == i].mean(axis=0) for i in range(self.n_classes)])

        # Loop over epochs to reconsider misclassified points and refine class centers
        for epoch in range(self.epoch):
            # Classify points based on current class centers
            classifications = self._classify_points(X)

            # Find misclassified points and apply secondary decision layers
            misclassified_points = self._secondary_decision_layers(X, classifications, y)

            # If no misclassified points, break and stop the epochs
            if len(misclassified_points) == 0:
                break  # Convergence

            # Update centroids (class centers) based on misclassified points
            self.centroids = self._update_centroids(X, classifications, y)

    def _classify_points(self, X):
        """
        Classify each data point based on the forces (attraction and repulsion) exerted by the class centers.

        X: Data points to classify

        Returns: Class predictions (0 to n_classes-1)
        """
        # Calculate distances between points and class centroids
        distances = cdist(X, self.centroids)
        total_forces = []

        # For each point, calculate the total force from all class centers
        for i, point in enumerate(X):
            forces = np.zeros(self.n_classes)
            for j in range(self.n_classes):
                dist_to_class = distances[i, j]
                forces[j] = self.magnetic_strength / (dist_to_class ** 2)

                # Repulsion from other classes (prevent overlap)
                for k in range(self.n_classes):
                    if k != j:
                        dist_to_other_class = distances[i, k]
                        forces[j] -= self.magnetic_strength / (dist_to_other_class ** 2)

            total_forces.append(forces)
        return np.argmax(total_forces, axis=1)

    def _secondary_decision_layers(self, X, classifications, y):
        """
        Apply secondary decision layers to both the repulsive and attractive layers to identify misclassified points.

        X: Data points
        classifications: Current predictions
        y: True labels

        Returns: List of misclassified points
        """
        misclassified_points = []

        # Apply the secondary decision layer to both repulsive and attractive layers
        for i in range(self.n_classes):
            if i == 0:  # Repulsive layer
                misclassified_points += list(X[(classifications != i) & (y == i)])
            else:  # Attractive layer
                misclassified_points += list(X[(classifications != i) & (y == i)])

        return misclassified_points

    def _update_centroids(self, X, classifications, y):
        """
        Update the class centroids based on the misclassified points and correct points.

        X: Data points
        classifications: Current predictions
        y: True labels

        Returns: Updated class centroids
        """
        new_centroids = self.centroids.copy()

        for i in range(self.n_classes):
            # Find misclassified points for the current class
            misclassified_points = X[(classifications != i) & (y == i)]
            correctly_classified_points = X[(classifications == i) & (y == i)]

            if len(misclassified_points) > 0:
                # Adjust the centroid to move toward correctly classified points and away from misclassified points
                new_centroids[i] = self.centroids[i] + self.learning_rate * (
                    np.mean(correctly_classified_points, axis=0) - np.mean(misclassified_points, axis=0)
                )

        return new_centroids

    def predict(self, X):
        """
        Predict the classes of new data points.

        X: New data points

        Returns: Predicted classes for the data points
        """
        return self._classify_points(X)

    def get_centroids(self):
        """
        Get the current class centroids (magnetic poles).
        """
        return self.centroids



# The MagneticClassifierWithRepulsion class is an extension of the MagneticInspiredClassifier that adds the concept of
# **inter-class repulsion** to the magnetic-inspired classification algorithm. This class simulates the behavior of magnetic
# forces between data points and class centers (magnetic poles), where the class centers attract points of their respective
# class and repel points from other classes. The core advantage of this class is that it not only adjusts the class centers
# based on misclassified points but also applies a repulsive force between class centers to ensure they remain well-separated,
# preventing overlap between the decision boundaries of different classes.
#
# Key features:
# 1. **Magnetic Forces**: The algorithm applies both **attraction** and **repulsion** forces.
#    - Attractive forces pull points toward their correct class centers.
#    - Repulsive forces push class centers apart if they are too close, ensuring clear separation.
# 2. **Epoch-Based Reconsideration**: The model is trained over multiple epochs, where each epoch involves reconsidering
#    misclassified points, refining the decision boundaries.
# 3. **Centroid Update**: The class centers are updated iteratively by moving them towards correctly classified points and
#    away from misclassified ones. The repulsive force between class centers helps maintain proper separation.
# 4. **Inter-Class Repulsion**: This class extends the base algorithm by introducing a **repulsion force** between class
#    centers to prevent overlap, making it more effective in handling complex or non-linearly separable data.
#
# Benefits:
# - Provides **dynamic classification** that adapts and refines the decision boundaries based on misclassified points.
# - The **repulsion mechanism** ensures that class centers do not overlap, making the decision boundaries clearer.
# - It offers a flexible approach to classification by using forces (magnetic-like) to manage both attraction and repulsion.
#
# Limitations:
# - The performance can be sensitive to the choice of parameters, such as the learning rate and repulsion strength.
# - It may not perform well with highly noisy or complex datasets without further optimization or enhancements.
#
# In summary, the MagneticClassifierWithRepulsion is an advanced version of the MagneticInspiredClassifier that incorporates
# repulsive forces to handle class overlap, making it a more robust and dynamic classification algorithm, especially for
# non-linearly separable data.


In [11]:
# Extended Class to Handle Inter-Class Repulsion (Prevent Overlap Between Class Centers)
class MagneticClassifierWithRepulsion(MagneticInspiredClassifier):
    def __init__(self, n_classes, max_iter=100, epoch=3, learning_rate=0.1, magnetic_strength=1.0, repel_strength=1.0):
        """
        Initialize the extended classifier with additional repulsion force between class centers.

        repel_strength: Strength of the repulsive force between class centers
        """
        super().__init__(n_classes, max_iter, epoch, learning_rate, magnetic_strength)
        self.repel_strength = repel_strength

    def _update_centroids(self, X, classifications, y):
        """
        Update the class centroids with an additional repulsive force to avoid overlap between class centers.

        X: Data points
        classifications: Current predictions
        y: True labels

        Returns: Updated class centroids with repulsion adjustments
        """
        new_centroids = self.centroids.copy()

        for i in range(self.n_classes):
            # Find misclassified points for the current class
            misclassified_points = X[(classifications != i) & (y == i)]
            correctly_classified_points = X[(classifications == i) & (y == i)]

            if len(misclassified_points) > 0:
                # Adjust the centroid based on misclassified points
                new_centroids[i] = self.centroids[i] + self.learning_rate * (
                    np.mean(correctly_classified_points, axis=0) - np.mean(misclassified_points, axis=0)
                )

            # Apply repulsion from other class centers (Inter-Class Interaction)
            for j in range(self.n_classes):
                if i != j:
                    # Calculate the repulsive force between class centers to push them apart
                    repulsion = (self.centroids[i] - self.centroids[j]) * self.repel_strength / np.linalg.norm(self.centroids[i] - self.centroids[j]) ** 2
                    new_centroids[i] -= repulsion

        return new_centroids


#Usage

In [55]:
def run_optimized_magnetic_classifier():
    # Generate a complex linear dataset
    X, y = make_classification(
        n_samples=10000,
        n_features=500,
        n_classes=2,
        n_informative=25,
        n_redundant=15,
        n_repeated=10,
        random_state=42
    )

    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape}")

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Dimensionality reduction using PCA
    pca = PCA(n_components=30)  # Reduce to 30 components
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    # Dynamically set epoch (10% of feature count)
    epoch = min(int(X_train.shape[1] * 0.1), 1000)
    print(f"Epochs set to: {epoch}")

    # Experiment with Learning Rate and Magnetic Strength & Repulsion Strength adjustments
    for learning_rate in [0.05]:  # Keep LR low for stability
        for magnetic_strength in [2.5]:  # Best performing magnetic strength
            for repel_strength in [2.5]:  # Best performing repulsion strength
                print(f"\nTesting with LR={learning_rate}, MS={magnetic_strength}, RS={repel_strength}")
                magnetic_classifier = MagneticClassifierWithRepulsion(
                    n_classes=2,
                    max_iter=100,
                    epoch=epoch,
                    learning_rate=learning_rate,
                    magnetic_strength=magnetic_strength,
                    repel_strength=repel_strength,
                )
                magnetic_classifier.fit(X_train, y_train)
                y_pred_magnetic = magnetic_classifier.predict(X_test)

                # --- Support Vector Machine ---
                svm_classifier = SVC(kernel='linear', random_state=42)
                svm_classifier.fit(X_train, y_train)
                y_pred_svm = svm_classifier.predict(X_test)

                # Evaluate accuracy for both classifiers
                accuracy_magnetic = accuracy_score(y_test, y_pred_magnetic)
                accuracy_svm = accuracy_score(y_test, y_pred_svm)

                print(f"Magnetic Classifier Accuracy (LR={learning_rate}, MS={magnetic_strength}, RS={repel_strength}): {accuracy_magnetic * 100:.2f}%")
                print(f"SVM Accuracy: {accuracy_svm * 100:.2f}%")

# Run the optimized Magnetic classifier comparison
run_optimized_magnetic_classifier()


Shape of X: (10000, 500)
Shape of y: (10000,)
Epochs set to: 3

Testing with LR=0.05, MS=2.5, RS=2.5
Magnetic Classifier Accuracy (LR=0.05, MS=2.5, RS=2.5): 74.77%
SVM Accuracy: 75.37%
